In [117]:
import sympy as sp
from sympy import *
from sympy.abc import *
from sympy.parsing.sympy_parser import parse_expr
import pdb
import itertools


In [118]:
def given(fx):
    li = [
        "1+-2.3+4+5.-6",
        "1.(2 + (3.5) + -1.3) + 4.6 + 12 + 15",
        "1.(2 + 3.5 + -1.3() + 4.6 + 12 + 15.16 + (17.18) + (-1)",
        "(1+(2).(3+5).(-1+3)).(4+6).(12).(15+16).((17+18)).(((-1)))",
        "1 . 2 . 3 . 4 . -5 . -1 . (-2 . (-1))",
        "1+-2.3+4+5+-6",
    ]
    li_ret = []
    for cl in li:
        try:
            li_ret.append([cl, fx(cl)])
        except Exception as e:
            print(e)
            li_ret.append('{e.message}')
    return li_ret
        

In [119]:
max_number = float('inf')

In [120]:
def max_no(clause):
    final = []
    clause = clause.replace('(', '')
    clause = clause.replace(')', '')
    dot_split = clause.split('.')
    for ds in dot_split:
        final.extend(ds.split('+'))
    for i in range(len(final)):
        final[i] = abs(int(final[i]))
    return max(final)



In [121]:
def define_variables(max_no):
    return symbols(f'x:{max_no}')

In [122]:
def replace_nums_with_symbols(clause):
    max_num = max_no(clause)
    clause = clause.replace('-', '~')
    split_by_and = clause.split('.')
    syms = define_variables(max_num+1)
    for i in range(max_num+1):
        exec(f'x{i} = syms[{i}]')
    fin_clause = ''
    for sub_clause in split_by_and:
        fin_sub_clause = ''
        for sub_sub_clause in sub_clause.split('+'):
            for i in range(max_num, 0, -1):
                if sub_sub_clause.replace(str(i), str(syms[i])) != sub_sub_clause:
                    sub_sub_clause = sub_sub_clause.replace(str(i), str(syms[i]))
                    break
            fin_sub_clause = fin_sub_clause + '|' + sub_sub_clause
        
        fin_clause = fin_clause + '&' + fin_sub_clause[1:]
    return fin_clause[1:]

In [123]:
given(replace_nums_with_symbols)

[['1+-2.3+4+5.-6', 'x1|~x2&x3|x4|x5&~x6'],
 ['1.(2 + (3.5) + -1.3) + 4.6 + 12 + 15',
  'x1&(x2 | (x3&x5) | ~x1&x3) | x4&x6 | x12 | x15'],
 ['1.(2 + 3.5 + -1.3() + 4.6 + 12 + 15.16 + (17.18) + (-1)',
  'x1&(x2 | x3&x5 | ~x1&x3() | x4&x6 | x12 | x15&x16 | (x17&x18) | (~x1)'],
 ['(1+(2).(3+5).(-1+3)).(4+6).(12).(15+16).((17+18)).(((-1)))',
  '(x1|(x2)&(x3|x5)&(~x1|x3))&(x4|x6)&(x12)&(x15|x16)&((x17|x18))&(((~x1)))'],
 ['1 . 2 . 3 . 4 . -5 . -1 . (-2 . (-1))',
  'x1 & x2 & x3 & x4 & ~x5 & ~x1 & (~x2 & (~x1))'],
 ['1+-2.3+4+5+-6', 'x1|~x2&x3|x4|x5|~x6']]

In [124]:
def auto_convert_to_cnf(clause):
    
    converted_clause_string = replace_nums_with_symbols(clause)
    try:
        exec(f'converted_clause = {converted_clause_string}')
        print(clause, sp.to_cnf(converted_clause_string))
        print()
    except:
        print(f'Error in clause {clause}')
        print()

In [125]:
def get_sp_clause(clause):
    converted_clause_string = replace_nums_with_symbols(clause)
    try:
        return parse_expr(converted_clause_string)
    except Exception as e:
        print(f'Error in clause {clause} {e}')
        print()

In [126]:
given(get_sp_clause)

Error in clause 1.(2 + 3.5 + -1.3() + 4.6 + 12 + 15.16 + (17.18) + (-1) ('EOF in multi-line statement', (2, 0))



[['1+-2.3+4+5.-6', x1 | x4 | (x3 & ~x2) | (x5 & ~x6)],
 ['1.(2 + (3.5) + -1.3) + 4.6 + 12 + 15',
  x12 | x15 | (x4 & x6) | (x1 & (x2 | (x3 & x5) | (x3 & ~x1)))],
 ['1.(2 + 3.5 + -1.3() + 4.6 + 12 + 15.16 + (17.18) + (-1)', None],
 ['(1+(2).(3+5).(-1+3)).(4+6).(12).(15+16).((17+18)).(((-1)))',
  x12 & ~x1 & (x15 | x16) & (x17 | x18) & (x4 | x6) & (x1 | (x2 & (x3 | x5) & (x3 | ~x1)))],
 ['1 . 2 . 3 . 4 . -5 . -1 . (-2 . (-1))',
  x1 & x2 & x3 & x4 & ~x1 & ~x2 & ~x5],
 ['1+-2.3+4+5+-6', x1 | x4 | x5 | ~x6 | (x3 & ~x2)]]

In [127]:
def subformulas(expr):
# subformula of expr without atoms
# considering operator as binary
    
    if expr.func == sp.Symbol:
        return []
    elif len(expr.args)==1:
        return [expr] + subformulas(expr.args[0])
    elif len(expr.args)==2:
        return [expr] + subformulas(expr.args[0]) + subformulas(expr.args[1]) 
    elif len(expr.args)>2:
        return [expr] + subformulas(expr.args[0]) + subformulas(eval(str(expr.func)+str(expr.args[1:])))  
    else:
        return "error"

In [128]:
syms = define_variables(18+1)
for i in range(18+1):
    exec(f'x{i} = syms[{i}]')

In [129]:
~x1

~x1

In [130]:
subformulas((x1 | x4 | (x3 & ~x2) | (x5 & ~x6)))

[x1 | x4 | (x3 & ~x2) | (x5 & ~x6),
 x4 | (x3 & ~x2) | (x5 & ~x6),
 (x3 & ~x2) | (x5 & ~x6),
 x3 & ~x2,
 ~x2,
 x5 & ~x6,
 ~x6]

In [131]:
def depth(expr):
# still considering operator as binary
    if expr.func == Not:
        return 1 + depth(expr.args[0])
    elif (len(expr.args)==2):
        return 1 + max([depth(x) for x in expr.args])
    elif ( len(expr.args)>2):
        #return 1 + max(depth(expr.args[0]), depth(eval(str(expr.func)+str(expr.args[1:]))))
        return len(expr.args)-1 + max([depth(x) for x in expr.args])
    elif expr.func == Symbol:
        return 0
    else: 
        return"error"

In [132]:
depth(x1 | x4 | (x3 & ~x2) | (x5 & ~x6))

5

In [133]:
F = x1 | x4 | (x3 & ~x2) | (x5 & ~x2)
sorted(set(subformulas(F)), key=depth)

[~x2,
 x5 & ~x2,
 x3 & ~x2,
 (x3 & ~x2) | (x5 & ~x2),
 x4 | (x3 & ~x2) | (x5 & ~x2),
 x1 | x4 | (x3 & ~x2) | (x5 & ~x2)]

In [134]:
def symbol_init(x, count=0):
    def new_symbol ():
        nonlocal count
        count += 1
        return var(str(x) + str(count)) #var add the new symbol in the name space
    return new_symbol

In [135]:
def binSubs(x, y, z):
    #replace y by z in x
    #print("subs", x, y, z)
    if (len(y.args) >0) and (len(x.args)>2) and (y!=x) and (y.func==x.func):
     
        if all([y in x.args for y in y.args]): 
            l = list(x.args)
            for i in y.args:
                l.remove(i)
            if (z.func== Symbol):
                t=tuple(l + [z])
            else:
                t=tuple(l+list(z.args))
            return eval(str(x.func) + str(t))
    
    return x.subs(y,z)

In [167]:
def tseitin1(s):
    # for binary operation
    try:
        
        expr = get_sp_clause(s)
        s=symbol_init("p")
        if expr.func == sp.Symbol:
            return true
        else:
            return tseitin_aux(sorted(set(subformulas(expr)), key=depth), s) 
    except Exception as e:
        raise e

def tseitin_aux(L, s):
    #nonlocal newvar
    newvar = s()
    if (L==[]):
        return true
    if depth(L[0]) > 0:
        if len(L[1:])>0 :
            #newList = [x.subs(L[0], newvar) for x in L[1:]]  
            
            newList = [binSubs(x, L[0], newvar) for x in L[1:]]  
            
            return Equivalent(newvar, L[0]) & tseitin_aux(newList, s)
        else:      
            return Equivalent(newvar, L[0]) & newvar
    else:
        print("error")
    

In [137]:
sp.to_cnf(tseitin1("1 . 2 . 3 . 4 . -5 . -1 . (-2 . (-1))"))

In [138]:
# given(tseitin1)

In [168]:
ans = tseitin1("1.(-2 + -1)")

In [169]:
ans

p4 & (Equivalent(p1, ~x2)) & (Equivalent(p2, ~x1)) & (Equivalent(p3, p1 | p2)) & (Equivalent(p4, p3 & x1))

In [141]:
given(tseitin1)

[['1+-2.3+4+5.-6', None],
 ['1.(2 + (3.5) + -1.3) + 4.6 + 12 + 15', None],
 ['1.(2 + 3.5 + -1.3() + 4.6 + 12 + 15.16 + (17.18) + (-1)', None],
 ['(1+(2).(3+5).(-1+3)).(4+6).(12).(15+16).((17+18)).(((-1)))', None],
 ['1 . 2 . 3 . 4 . -5 . -1 . (-2 . (-1))', None],
 ['1+-2.3+4+5+-6', None]]

In [142]:
ans

In [143]:
sp.to_cnf(ans)

In [144]:
def cnf(a, neg=False, depth=0):
#     print(depth*'\t', a, a.func, len(a.args), neg)
    if a.func == sp.Symbol:
        if neg:
            return ~a
        else:
            return a
    elif len(a.args)==1:
#         print(a.args)
        return cnf(eval(str(a.args[0])), neg=not neg, depth=depth+1)
    elif len(a.args)==2 and a.func == sp.Equivalent:
        if not neg:
            return demorgan(cnf(a.args[0], depth=depth+1) | cnf(a.args[1], neg=True, depth=depth+1), depth=depth+1) & demorgan(cnf(a.args[0], neg=True, depth=depth+1) | cnf(a.args[1], depth=depth+1), depth=depth+1)
        else:
            assert False
            return (cnf(a.args[0], neg=True, depth=depth+1) | cnf(a.args[1], neg=True, depth=depth+1) & (cnf(a.args[1], neg=False, depth=depth+1) | cnf(a.args[0], neg=False, depth=depth+1)))
    elif len(a.args)>=2:
        if a.func == sp.And:
            if not neg:
                return cnf(a.args[0], depth=depth+1) & cnf(eval(str(a.func)+str(a.args[1:])), depth=depth+1)  
            else:
                return (cnf(a.args[0], neg=True, depth=depth+1) | cnf(eval(str(a.func)+str(a.args[1:])), neg=True, depth=depth+1))
        elif a.func == sp.Or:
            if not neg:
                return demorgan(cnf(a.args[0], depth=depth+1) | cnf(eval(str(a.func)+str(a.args[1:])), depth=depth+1), depth=depth+1)
            else:
                return cnf(a.args[0], depth=depth+1, neg=neg) & cnf(eval(str(a.func)+str(a.args[1:])), neg=neg, depth=depth+1)
    else:
        return "error"

In [160]:
def cnf(a, neg=False, depth=0):
    print(depth*'\t', a, a.func, len(a.args), neg)
    if a.func == sp.Symbol:
        if neg:
            return ~a
        else:
            return a
    elif len(a.args)==1:
#         print(a.args)
        return cnf(eval(str(a.args[0])), neg=not neg, depth=depth+1)
    elif len(a.args)==2 and a.func == sp.Equivalent:
        if not neg:
            return demorgan(cnf(a.args[0], depth=depth+1) | cnf(a.args[1], neg=True, depth=depth+1), depth=depth+1) & demorgan(cnf(a.args[0], neg=True, depth=depth+1) | cnf(a.args[1], depth=depth+1), depth=depth+1)
        else:
            assert False
            # return (cnf(a.args[0], neg=True, depth=depth+1) | cnf(a.args[1], neg=True, depth=depth+1) & (cnf(a.args[1], neg=False, depth=depth+1) | cnf(a.args[0], neg=False, depth=depth+1)))
    elif len(a.args)>=2:
        if a.func == sp.And:
            if not neg:
                return cnf(a.args[0], depth=depth+1) & demorgan(cnf(eval(str(a.func)+str(a.args[1:])), depth=depth+1), depth=depth+1)
            else:
                return demorgan(cnf(a.args[0], neg=True, depth=depth+1) | demorgan(cnf(eval(str(a.func)+str(a.args[1:])), neg=True, depth=depth+1), depth=depth+1), depth=depth+1)
        elif a.func == sp.Or:
            if not neg:
                return demorgan(cnf(a.args[0], depth=depth+1) | cnf(eval(str(a.func)+str(a.args[1:])), depth=depth+1), depth=depth+1)
            else:
                return cnf(a.args[0], depth=depth+1, neg=neg) & demorgan(cnf(eval(str(a.func)+str(a.args[1:])), neg=neg, depth=depth+1), depth=depth+1)
    else:
        return "error"


In [230]:
def demorgan(a, neg=False, depth=0):
#     print(depth*"\t","demorgan", a)
    s = f''
#     print(depth*"\t", a)
    if a.func == sp.Or:
        ls = []
        for i in range(len(a.args)):
            if a.args[i].func == sp.And:
    #             print(depth*"\t",'1')
                ls.append(a.args[i].args)
            elif a.args[i].func == sp.Symbol:
                ls.append([a.args[i]])
            else:
                return a
        ls2 = set()
        for element in itertools.product(*ls):
            ls2.add('And' + str(tuple(set(element))))
        ls2_str = 'Or(' + ','.join(ls2) + ')'
        print('ls2', ls2)
        return parse_expr(ls2_str)
    else:
        return a

In [240]:
demorgan(p1 | (p2 & p3)| (p2 & p3)| (p2 & p3)| (p5 & p3)| (p4 & p3)| (p9 & p3)| (p1 & p3)| (p10 & p3)| (p2 & p3)| (p2 & p5)| (p2 & p3)| (p12 & p7)| (p10 & p11)| (p11 & p3)| (p2 & p3)| (p2 & p3)| (p2 & p3)| (p2 & p3)| (p2 & p3)| (p2 & p9))
# Or(p4, p3, ~p5 & (x12 & x2 & x8))

[[p1], (p1, p3), (p10, p11), (p10, p3), (p11, p3), (p12, p7), (p2, p3), (p2, p5), (p2, p9), (p3, p4), (p3, p5), (p3, p9)]
Or(And(p1, p1, p10, p10, p11, p12, p2, p2, p2, p3, p3, p3),And(p1, p1, p10, p10, p11, p12, p2, p2, p2, p3, p3, p9),And(p1, p1, p10, p10, p11, p12, p2, p2, p2, p3, p5, p3),And(p1, p1, p10, p10, p11, p12, p2, p2, p2, p3, p5, p9),And(p1, p1, p10, p10, p11, p12, p2, p2, p2, p4, p3, p3),And(p1, p1, p10, p10, p11, p12, p2, p2, p2, p4, p3, p9),And(p1, p1, p10, p10, p11, p12, p2, p2, p2, p4, p5, p3),And(p1, p1, p10, p10, p11, p12, p2, p2, p2, p4, p5, p9),And(p1, p1, p10, p10, p11, p12, p2, p2, p9, p3, p3, p3),And(p1, p1, p10, p10, p11, p12, p2, p2, p9, p3, p3, p9),And(p1, p1, p10, p10, p11, p12, p2, p2, p9, p3, p5, p3),And(p1, p1, p10, p10, p11, p12, p2, p2, p9, p3, p5, p9),And(p1, p1, p10, p10, p11, p12, p2, p2, p9, p4, p3, p3),And(p1, p1, p10, p10, p11, p12, p2, p2, p9, p4, p3, p9),And(p1, p1, p10, p10, p11, p12, p2, p2, p9, p4, p5, p3),And(p1, p1, p10, p10, p11, p12, p2,

SyntaxError: more than 255 arguments (<string>, line 1)

In [162]:
ans

In [161]:
cnf(ans)

AttributeError: 'NoneType' object has no attribute 'func'

In [149]:
def make_cnf(s):
    clause = tseitin1(s)
    cnf_ans = cnf(clause)
    return cnf_ans

In [150]:
li = [
#         "1+-2.3+4+5.-6",
#         "1.(2 + (3.5) + -1.3) + 4.6 + 12 + 15",
#         "1.(2 + 3.5 + -1.3() + 4.6 + 12 + 15.16 + (17.18) + (-1)",
        "(1+(2).(3+5).(-1+3)).(4+6).(12).(15+16).((17+18)).(((-1)))",
#         "1 . 2 . 3 . 4 . -5 . -1 . (-2 . (-1))",
#         "1+-2.3+4+5+-6",
#         "1.2.3",
#         "1+2.3",
#         "1+-3",
        "7+-4.-7.-1.-9+-4.-2"
    ]
for clause in li:
    tseitin_clause = tseitin1(clause)
    print(tseitin_clause)
    if tseitin_clause is not None:
        sp_cnf = sp.to_cnf(tseitin_clause)
        cnf_ans = cnf(tseitin_clause)
        print(sp_cnf == cnf_ans)

None
None


In [151]:
cnf_ans

p11 & (p1 | x7) & (p2 | x4) & (p3 | x2) & (p4 | x1) & (p5 | x9) & (p1 | ~p6) & (p10 | ~p7) & (p11 | ~p10) & (p11 | ~x7) & (p2 | ~p7) & (p2 | ~p8) & (p3 | ~p7) & (p4 | ~p9) & (p5 | ~p6) & (p6 | ~p8) & (p8 | ~p9) & (~p1 | ~x7) & (~p2 | ~x4) & (~p3 | ~x2) & (~p4 | ~x1) & (~p5 | ~x9) & (p1 | p7 | ~p10) & (p10 | x7 | ~p11) & (p2 | p7 | ~p10) & (p4 | p7 | ~p10) & (p5 | p7 | ~p10) & (p6 | ~p1 | ~p5) & (p7 | ~p2 | ~p3) & (p8 | ~p2 | ~p6) & (p9 | ~p4 | ~p8) & (p10 | ~p1 | ~p2 | ~p4 | ~p5)

In [152]:
def set_wrapper(cnf_ans):
    set_of_vars = {}
    counter = 1
    def recursive_add_to_set(a):
        nonlocal counter
        if a.func == sp.Symbol:
            if a not in set_of_vars:
                set_of_vars[a] = counter
                counter += 1
        elif len(a.args)==1:
            b = a.args[0]
            if b not in set_of_vars:
                set_of_vars[b] = counter
                counter += 1
        else:
            for arg in a.args:
                recursive_add_to_set(arg)
    recursive_add_to_set(cnf_ans)
    return set_of_vars
set_of_vars = set_wrapper(cnf_ans)

In [153]:
def write_cnf(filename):
    assert cnf_ans.func == sp.And
    l = [f'p cnf {len(set_of_vars)} {len(cnf_ans.args)}']
    for arg in cnf_ans.args:
        s = ''
        if arg.func == sp.Symbol:
            s += str(set_of_vars[arg]) + ' '
        else:
            assert arg.func == sp.Or
            for sub_arg in arg.args:
                if len(sub_arg.args) == 1:
                    s += str(-1*(set_of_vars[sub_arg.args[0]])) + ' '
                elif len(sub_arg.args) == 0:
                    s += str(set_of_vars[sub_arg]) + ' '
        l.append(s)
    with open(filename,'w') as f:
        f.write('\n'.join(l))
    

In [154]:
write_cnf('test.cnf')

In [155]:
tseitin1('7+-4.-7.-1.-9+-4.-2')

In [156]:
len(cnf_ans.args)

32

In [157]:
cnf_ans.func

And

In [158]:
cnf_ans.args

(p11,
 p1 | x7,
 p2 | x4,
 p3 | x2,
 p4 | x1,
 p5 | x9,
 p1 | ~p6,
 p10 | ~p7,
 p11 | ~p10,
 p11 | ~x7,
 p2 | ~p7,
 p2 | ~p8,
 p3 | ~p7,
 p4 | ~p9,
 p5 | ~p6,
 p6 | ~p8,
 p8 | ~p9,
 ~p1 | ~x7,
 ~p2 | ~x4,
 ~p3 | ~x2,
 ~p4 | ~x1,
 ~p5 | ~x9,
 p1 | p7 | ~p10,
 p10 | x7 | ~p11,
 p2 | p7 | ~p10,
 p4 | p7 | ~p10,
 p5 | p7 | ~p10,
 p6 | ~p1 | ~p5,
 p7 | ~p2 | ~p3,
 p8 | ~p2 | ~p6,
 p9 | ~p4 | ~p8,
 p10 | ~p1 | ~p2 | ~p4 | ~p5)

In [159]:
set_of_vars

{p11: 1,
 p1: 2,
 x7: 3,
 p2: 4,
 x4: 5,
 p3: 6,
 x2: 7,
 p4: 8,
 x1: 9,
 p5: 10,
 x9: 11,
 p6: 12,
 p10: 13,
 p7: 14,
 p8: 15,
 p9: 16}